# Translating Roman numerals to integer

Train a neural network to translate roman numerals to integer, assuming a max length of 8. This is to demonstrate the capability of LSTM

### Step 1: an algorithmic way to predict integer from roman numerals. This requires a human understanding of how roman numerals work, and a CompSci 101 level programmer

In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)
from collections import OrderedDict

def write_roman(num):

  roman = OrderedDict()
  roman[1000] = "M"
  roman[900] = "CM"
  roman[500] = "D"
  roman[400] = "CD"
  roman[100] = "C"
  roman[90] = "XC"
  roman[50] = "L"
  roman[40] = "XL"
  roman[10] = "X"
  roman[9] = "IX"
  roman[5] = "V"
  roman[4] = "IV"
  roman[1] = "I"

  def roman_num(num):
      for r in roman.keys():
          x, y = divmod(num, r)
          yield roman[r] * x
          num -= (r * x)
          if num > 0:
              roman_num(num)
          else:
              break

  return "".join([a for a in roman_num(num)])

roman_values = []
ha_values = []
for i in range(10000):
  ha_values.append(str(i))
  roman_values.append(str(write_roman(i)))

roman_values = pd.Series(roman_values, dtype=str)
ha_values = pd.Series(ha_values, dtype=str)

In [14]:
df = pd.DataFrame({"roman":roman_values, "integer":ha_values})
df

,roman,integer
0,,0
1,I,1
2,II,2
3,III,3
4,IV,4
...,...,...
9995,MMMMMMMMMCMXCV,9995
9996,MMMMMMMMMCMXCVI,9996
9997,MMMMMMMMMCMXCVII,9997
9998,MMMMMMMMMCMXCVIII,9998


In [15]:
# shuffle the DataFrame rows
df = df.sample(frac = 1)
df

,roman,integer
6366,MMMMMMCCCLXVI,6366
9628,MMMMMMMMMDCXXVIII,9628
5916,MMMMMCMXVI,5916
681,DCLXXXI,681
5459,MMMMMCDLIX,5459
...,...,...
1690,MDCXC,1690
625,DCXXV,625
7523,MMMMMMMDXXIII,7523
5830,MMMMMDCCCXXX,5830


In [16]:
roman_values = df["roman"]
ha_values = df["integer"]

### Step 2: Set up an LSTM to learn the essence of Roman numerals. An RNN won't work since the neural network needs to be able to "look back" to interpret roman numerals

In [17]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

roman_tokenizer = Tokenizer(char_level=True)
roman_tokenizer.fit_on_texts(roman_values)

ha_tokenizer = Tokenizer(char_level=True)
ha_tokenizer.fit_on_texts(ha_values)
roman_index_to_char = {v: k for k,v in roman_tokenizer.word_index.items()}
ha_index_to_char = {v:k for k,v in ha_tokenizer.word_index.items()}

# Non Spatial Inputs
x_roman_1 = roman_tokenizer.texts_to_matrix(roman_values)
x_roman_2 = roman_tokenizer.texts_to_matrix(roman_values, mode='count')
# Spatial Inputs
x_roman_3 = pad_sequences(roman_tokenizer.texts_to_sequences(roman_values), padding='pre')


num_ex = x_roman_3.shape[0]
roman_seq_len = x_roman_3.shape[1]
roman_vocab_size = len(roman_index_to_char) + 1


arr = [to_categorical(x_roman_3[i], num_classes=roman_vocab_size) for i in range(num_ex)]
arr = [item.reshape(roman_seq_len * roman_vocab_size,) for item in arr]
temp = np.vstack(arr)

x_roman_3 = temp.reshape((num_ex,roman_seq_len,roman_vocab_size))
x_roman = np.hstack([x_roman_1, x_roman_2])

In [18]:
y_ha = ha_tokenizer.texts_to_sequences(ha_values)
y_ha = pad_sequences(y_ha, padding='post')

ha_seq_len = y_ha.shape[1]
ha_vocab_size = len(ha_index_to_char) + 1

arr = [to_categorical(y_ha[i], num_classes=ha_vocab_size) for i in range(num_ex)]
arr = [item.reshape(ha_seq_len * ha_vocab_size,) for item in arr]
temp = np.vstack(arr)
y_ha = temp.reshape((num_ex,ha_seq_len,ha_vocab_size))


from keras.models import Sequential, Model
from keras.layers import Dense, Activation, SimpleRNN, LSTM, RepeatVector, Flatten,  Input, Reshape

NON_SPATIAL_INPUT_DIM = x_roman.shape[1]

input1_nonspatial = Input(shape=(NON_SPATIAL_INPUT_DIM,))

input2_spatial = Input(shape=(roman_seq_len, roman_vocab_size))
intm = LSTM(20)(input2_spatial)
intm = RepeatVector(ha_seq_len)(intm)
intm = LSTM(20, return_sequences=True)(intm)
intm = LSTM(20, return_sequences=True)(intm)
rnn_output = Flatten()(intm)

concatenated = keras.layers.concatenate([input1_nonspatial, rnn_output])

intm = Dense(ha_vocab_size*ha_seq_len*2)(concatenated)
intm = Activation('relu')(intm)

intm = Dense(ha_vocab_size*ha_seq_len)(intm)
intm = Reshape(target_shape=(ha_seq_len, ha_vocab_size))(intm)
output = Activation('softmax')(intm)

model = Model(inputs=[input1_nonspatial, input2_spatial], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(x=[x_roman, x_roman_3], y=y_ha, epochs=1000, validation_split=0.3, batch_size=10)

Epoch 1/1000
700/700 [==============================] - 13s 13ms/step - loss: 1.5827 - accuracy: 0.4735 - val_loss: 1.1581 - val_accuracy: 0.6439
Epoch 2/1000
700/700 [==============================] - 8s 11ms/step - loss: 0.9594 - accuracy: 0.6829 - val_loss: 0.8197 - val_accuracy: 0.7283
Epoch 3/1000
700/700 [==============================] - 8s 11ms/step - loss: 0.7134 - accuracy: 0.7526 - val_loss: 0.6100 - val_accuracy: 0.7964
Epoch 4/1000
700/700 [==============================] - 8s 11ms/step - loss: 0.5549 - accuracy: 0.7986 - val_loss: 0.5223 - val_accuracy: 0.7985
Epoch 5/1000
700/700 [==============================] - 8s 11ms/step - loss: 0.4580 - accuracy: 0.8229 - val_loss: 0.4093 - val_accuracy: 0.8306
Epoch 6/1000
700/700 [==============================] - 8s 11ms/step - loss: 0.3889 - accuracy: 0.8417 - val_loss: 0.3675 - val_accuracy: 0.8432
Epoch 7/1000
700/700 [==============================] - 8s 11ms/step - loss: 0.3307 - accuracy: 0.8621 - val_loss: 0.2958 - val_a

KeyboardInterrupt: ignored